In [1]:
%matplotlib inline
import sys
print(sys.version)
import numpy as np
print(np.__version__)
import pandas as pd
print(pd.__version__)
import matplotlib.pyplot as plt
from pandas_datareader import data, wb
from datetime import timedelta

3.5.1 |Anaconda 2.4.1 (x86_64)| (default, Dec  7 2015, 11:24:55) 
[GCC 4.2.1 (Apple Inc. build 5577)]
1.11.0
0.18.0


In [2]:
def fixtemp(value):
    if (value == ' '):
        value = np.nan
    elif float(value) < 20.0:
        value = np.nan
    elif type(value) == str:
        value = float(value)
    return value

def fixhumid(value):
    if float(value) > 100.0:
        value = np.nan
    return value

def fixbar(value):
    if value > 35:
        value = value/10
    return value * 100

def makestring(value):
    return str(value)[0:11]

def changetime(value):
    zonemap = {'EDT': 5, 'CDT': 6, 'MDT': 7, 'PDT': 8, 'MST': 8}
    return timedelta(hours=zonemap[value])
    
def getstation(remarkstring):
    for position in range(len(remarkstring)):
        if remarkstring[position] == "K":
            return remarkstring[position:position+4]

all_cols = ['month', 'day', 'year', 'hour', 'minute', 'timezone', 'dirtytemp', 'dirtyhumidity', 'windspKts', \
                'gustKts', 'winddir', 'qualchkflag', 'sealvlpresIn', 'dirtybar', \
                'preschg', 'weathercond', 'visibilMi', 'remark', 'stationID', 'dewptF']
keep_cols = ['month', 'day', 'year', 'hour', 'minute', 'timezone', 'dirtytemp', 'dirtyhumidity', \
             'dirtybar', 'stationID']

weather = pd.read_csv("MLB_Weather_Data.csv", header=None, names=all_cols, usecols=keep_cols, \
                     dtype={'timezone': str, 'year':int, 'month':int, 'day':int, 'hour':int, 'minute':int})

weather['timestamp'] = pd.to_datetime(weather['year'].apply(makestring) + "-" + \
                        weather['month'].apply(makestring) + "-" + \
                        weather['day'].apply(makestring) + " " + \
                        weather['hour'].apply(makestring) + ":" + \
                        weather['minute'].apply(makestring) + ":00")
weather['gmt'] = weather.timestamp + weather.timezone.apply(changetime)
weather['stationID'] = weather.stationID.apply(getstation)
weather['tempF'] = weather.dirtytemp.apply(fixtemp).interpolate()
weather.tempF = weather.tempF.apply(int)
weather['relhum'] = weather.dirtyhumidity.apply(fixhumid).interpolate()
weather.relhum = weather.relhum.apply(int)
weather['altibarom'] = weather.dirtybar.apply(fixbar).interpolate()
weather.altibarom = weather.altibarom.apply(int)/100
del weather['month']
del weather['day']
del weather['year']
del weather['hour']
del weather['minute']
del weather['timezone']
del weather['timestamp']
del weather['dirtytemp']
del weather['dirtyhumidity']
del weather['dirtybar']
weather.set_index('gmt')

/Users/pyvarjan/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,stationID,tempF,relhum,altibarom
gmt,,,,
2015-10-11 04:51:00,KLGA,57,62,30.07
2015-10-11 03:51:00,KLGA,57,57,30.08
2015-10-11 02:51:00,KLGA,57,55,30.08
2015-10-11 01:51:00,KLGA,59,48,30.07
2015-10-11 00:51:00,KLGA,60,46,30.09
2015-10-10 23:51:00,KLGA,61,36,30.06
2015-10-10 22:51:00,KLGA,63,34,30.06
2015-10-10 21:51:00,KLGA,63,35,30.06
2015-10-10 20:51:00,KLGA,64,34,30.07


In [3]:
weather.dtypes

stationID            object
gmt          datetime64[ns]
tempF                 int64
relhum                int64
altibarom           float64
dtype: object

In [4]:
weather.stationID.unique()

array(['KLGA', 'KNYC', 'KBOS', 'KPHL', 'KBWI', 'KDCA', 'KATL', 'KSPG',
       'KMIA', 'KMKC', 'KMSP', 'KMDW', 'KBKL', 'KORD', 'KMKE', 'KDET',
       'KCPS', 'KPIT', 'KLUK', 'KDAL', 'KDEN', 'KCQT', 'KBFI', 'KSAN',
       'KOAK', 'KEFD', 'KPHX', 'KSFO', 'KSLI'], dtype=object)

In [ ]:
stationmap = {'NYY':'KNYC', 'BOS':'KBOS', 'BAL':'KBWI', 'TAM':'KSPG',
             'NYM':'KLGA', 'ATL':'KATL', 'PHI':'KPHL', 'WAS':'KDCA', 'MIA':'KMIA',
             'CWS':'KMDW', 'CLE':'KBKL', 'DET':'KDET', 'KCR':'KMKC', 'MIN':'KMSP',
             'CWC':'KORD', 'CIN':'KLUK', 'MIL':'KMKE', 'PIT':'KPIT', 'STL':'KCPS',
             'HOU':'KEFD', 'ANA':'KSLI', 'OAK':'KOAK', 'SEA':'KBFI', 'TEX':'KDAL',
             'ARI':'KPHX', 'COL':'KDEN', 'LAD':'KCQT', 'SDP':'KSAN', 'SFG':'KSFO'
             }

In [5]:
weather.tempF.unique()

array([ 57,  59,  60,  61,  63,  64,  62,  55,  53,  52,  66,  69,  75,
        78,  73,  70,  68,  71,  72,  54,  51,  50,  48,  77,  81,  80,
        79,  82,  84,  86,  87,  88,  90,  93,  91,  89,  46,  45,  42,
        39,  41,  44,  43,  56,  37,  36,  34,  35,  33,  96,  95,  32,
        30,  76,  97,  28,  27,  26,  38,  31,  29,  40,  98, 100,  99,
       102, 104, 105, 107, 106,  92,  67,  65, 108, 111, 109, 113, 114,
       116,  74,  83,  94, 103, 101,  85,  58,  49,  47])

In [6]:
weather.relhum.unique()

array([ 62,  57,  55,  48,  46,  36,  34,  35,  37,  38,  40,  44,  49,
        51,  56,  61,  59,  63,  60,  65,  67,  70,  75,  79,  81,  84,
        83,  87,  73,  54,  64,  58,  66,  71,  76,  86,  80,  78,  72,
        52,  69,  53,  47,  77,  45,  50,  74,  82,  89,  90,  93,  68,
        88,  85,  94,  91,  43,  42,  39,  31,  32,  29,  33,  41,  30,
        28,  27,  26,  25,  96, 100,  92,  19,  20,  17,  23,  97,  15,
        22,  24,  18,  16,  21,  14,  13,  12,  95,  99,  98,   9,   8,
        10,  11,   7,   6,   5,   4,   3,   2])

In [7]:
weather.altibarom.unique()

array([ 30.07,  30.08,  30.09,  30.06,  30.11,  30.12,  30.13,  30.05,
        30.01,  29.97,  29.95,  29.93,  29.89,  29.87,  29.86,  29.84,
        29.82,  29.8 ,  29.78,  29.77,  29.74,  29.76,  29.92,  29.99,
        30.  ,  30.02,  30.03,  30.1 ,  30.14,  30.15,  30.04,  29.98,
        29.94,  29.96,  30.17,  30.18,  30.2 ,  30.21,  30.22,  30.19,
        30.23,  30.24,  30.25,  30.26,  30.27,  30.28,  30.31,  30.33,
        30.34,  30.35,  30.32,  30.3 ,  30.29,  30.16,  29.85,  29.83,
        29.81,  29.73,  29.71,  29.69,  29.68,  29.67,  29.66,  29.9 ,
        29.91,  30.36,  30.39,  30.4 ,  30.42,  30.44,  30.46,  30.47,
        30.45,  30.48,  30.43,  30.41,  30.38,  30.37,  29.88,  29.72,
        29.7 ,  29.65,  29.64,  29.79,  29.75,  29.63,  29.61,  29.6 ,
        29.58,  29.57,  29.59,  29.62,  29.56,  29.55,  29.53,  29.54,
        30.49,  29.51,  29.48,  29.46,  29.5 ,  29.52,  30.5 ,  30.51,
        29.49,  30.52,  30.53,  29.45,  29.42,  29.41,  29.43,  29.4 ,
      

In [8]:
weather.to_csv('Clean_Weather.csv')